In [1]:
import nibabel as nib
import ants
from ants.plotting import plot
import numpy as np


In [ ]:
#read mapped image
mapped_image_path = '/Users/dennis.jungchildmind.org/Desktop/subcortical_test/I38_new_confidence/warped_template.nii.gz'
mapped_image = ants.image_read(mapped_image_path)
print(mapped_image)


In [ ]:
mapped_image_np = mapped_image.numpy()

In [ ]:
print(mapped_image_np.shape)

In [ ]:
print(np.unique(mapped_image_np))

In [ ]:
original_image_path = '/Users/dennis.jungchildmind.org/Desktop/subcortical_test/I38_new_confidence/orig_img_resampled.nii'
original_image = ants.image_read(original_image_path)
print(original_image)
plot(original_image,axis=2)

In [ ]:
tmp = nib.load(template_path)

In [ ]:
from matplotlib import pyplot as plt
print(tmp.get_fdata().shape)
print(np.unique(tmp.get_fdata()).shape)


In [ ]:
template_path = '/Users/dennis.jungchildmind.org/Desktop/atlas/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_t1_tal_nlin_asym_09c.nii'
headmask_path = '/Users/dennis.jungchildmind.org/Desktop/atlas/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_t1_tal_nlin_asym_09c_mask.nii'#only has t
template = ants.image_read(template_path)
headmask = ants.image_read(headmask_path)
masked_template = ants.mask_image(template, headmask,level=1,binarize=False)#this removes the skull right away
# Get numpy array
data = masked_template.numpy()

# Create left hemisphere mask
# In MNI space with ANTs, need to check orientation
center_x = data.shape[0] // 2
left_mask = np.zeros_like(data)
left_mask[:center_x, :, :] = 1

# Create ANTs image from mask
mask_img = ants.from_numpy(left_mask, origin=masked_template.origin, 
                           spacing=masked_template.spacing, direction=masked_template.direction)
left_hemisphere = masked_template * mask_img
plot(left_hemisphere,axis=1)
# Save result
ants.image_write(left_hemisphere, 'mni_icbm152_t1_tal_nlin_asym_09c_left_hemi.nii')

#do for right hemisphere
right_mask = np.zeros_like(data)
right_mask[center_x:, :, :] = 1
right_hemisphere = masked_template * ants.from_numpy(right_mask, origin=masked_template.origin, 
                           spacing=masked_template.spacing, direction=masked_template.direction)
plot(right_hemisphere,axis=1)
ants.image_write(right_hemisphere, 'mni_icbm152_t1_tal_nlin_asym_09c_right_hemi.nii')

In [ ]:


# Define file paths
template_path = '/Users/dennis.jungchildmind.org/Desktop/atlas/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_t2_tal_nlin_asym_09c.nii'
headmask_path = '/Users/dennis.jungchildmind.org/Desktop/atlas/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_t1_tal_nlin_asym_09c_mask.nii'#only has t
atlas_path = '/Users/dennis.jungchildmind.org/Desktop/subcortical_test/Schaefer2018_400Parcels_7Networks_order_Tian_Subcortex_S1_3T_MNI152NLin2009cAsym_1mm.nii.gz'

# Read images
template = ants.image_read(template_path)
headmask = ants.image_read(headmask_path)
atlas = ants.image_read(atlas_path)

#make sure the shape of the atlas and template are the same
if atlas.shape != template.shape:
    print("The shape of the atlas and template are not the same")
    exit()
else:
    print("The shape of the atlas and template are the same")


In [ ]:
masked_template = ants.mask_image(template, headmask,level=1,binarize=False)#this removes the skull right away
plot(masked_template,axis=2)
plot(atlas,axis=2)

In [ ]:
#get labels from the atlas so that I can generate a mask for the left or right hemisphere
atlas_label = atlas.numpy()
#just manual separate of lh and rh 
# Fix: Combine hardcoded values with range for RH labels according to the comment
rh_label = list(range(1, 9)) + list(range(217, 417))
lh_label = list(range(9, 217))  # 9 through 216 inclusive

#generate a mask for the left hemisphere
left_hemisphere_mask = np.isin(atlas_label, lh_label)
right_hemisphere_mask = np.isin(atlas_label, rh_label)
# Convert to ANTs images
# Method 1: Using from_numpy (RECOMMENDED)
left_hemi_mask_ants = ants.from_numpy(
    left_hemisphere_mask.astype('float32'),  # Convert bool to float
    origin=atlas.origin,
    spacing=atlas.spacing,
    direction=atlas.direction
)

right_hemi_mask_ants = ants.from_numpy(
    right_hemisphere_mask.astype('float32'),
    origin=atlas.origin,
    spacing=atlas.spacing,
    direction=atlas.direction
)


In [ ]:
masked_template_lh = ants.mask_image(masked_template,left_hemi_mask_ants,level=1,binarize=False)
plot(masked_template_lh,axis=2)
masked_template_rh = ants.mask_image(masked_template,right_hemi_mask_ants,level=1,binarize=False)
plot(masked_template_rh,axis=2)

#save the masked templates
# Save the masks
ants.image_write(masked_template_lh, 'mni_icbm152_t2_tal_nlin_asym_09c_masked_lh_only.nii.gz')
ants.image_write(masked_template_rh, 'mni_icbm152_t2_tal_nlin_asym_09c_masked_rh_only.nii.gz')

print(f"Left hemisphere mask: {left_hemi_mask_ants.shape}")
print(f"Right hemisphere mask: {right_hemi_mask_ants.shape}")
print(f"Left hemisphere voxels: {np.sum(left_hemisphere_mask)}")
print(f"Right hemisphere voxels: {np.sum(right_hemisphere_mask)}")

In [17]:
print(np.array(dists_dict['dist_lh']))
print(np.array(dists_dict['label']))


[        nan 43.17040495 45.49635934 27.22282983 20.5721383  42.28919479
 34.69781011 42.19293438 38.51152827 26.79548571 32.63160857 10.1579598
  4.61876665 35.47979882 19.18625484 25.7091647  29.84380739 43.03754274
 35.33918616 56.24260294 42.04502685 58.89166708 72.24595763 27.7690414
 64.72923622 72.03813689 44.65383935 59.05703344 82.23463442 27.44741097
 76.18864038 80.68663779 39.36571949 66.2874954  70.74577119 79.41141228
 31.18349471 56.60987574 47.37821486 72.60733547 79.42190782 69.0794426
 55.19709977 70.15999031 73.00640895 76.61737237 69.86386435 73.14514234
 42.52455668 46.98636966 27.13232676 31.0549871  30.19653251 53.59332156
 36.19690075 40.95275916 53.59547952 40.84026802 53.68958106 55.70575286
 59.85947819 50.75139931 57.59001885 56.9255993  52.10064949 40.96213839
 52.44423685 62.37445128 65.68359929 55.86807572 51.96142655 51.00856329
 54.48740379 63.47207078 66.93354088 61.78242829 65.70614992 61.65689369
 62.24928045 68.82978459 63.96824179 67.48086643 65.08